In [1]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [30]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms


In [26]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.pool = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(32*99*99,64)
        self.fc2 = nn.Linear(64,1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [59]:
model = CNN()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [60]:
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
            Linear-4                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.96
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------


In [61]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [62]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


train_dataset = ImageFolder(root='./data/train', transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

validation_dataset = ImageFolder(root='./data/test', transform=train_transforms)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)

In [63]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def train_model():
  num_epochs = 10
  history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0
      correct_train = 0
      total_train = 0
      for images, labels in train_loader:
          images, labels = images.to(device), labels.to(device)
          labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item() * images.size(0)
          # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
          predicted = (torch.sigmoid(outputs) > 0.5).float()
          total_train += labels.size(0)
          correct_train += (predicted == labels).sum().item()

      epoch_loss = running_loss / len(train_dataset)
      epoch_acc = correct_train / total_train
      history['loss'].append(epoch_loss)
      history['acc'].append(epoch_acc)

      model.eval()
      val_running_loss = 0.0
      correct_val = 0
      total_val = 0
      with torch.no_grad():
          for images, labels in validation_loader:
              images, labels = images.to(device), labels.to(device)
              labels = labels.float().unsqueeze(1)

              outputs = model(images)
              loss = criterion(outputs, labels)

              val_running_loss += loss.item() * images.size(0)
              predicted = (torch.sigmoid(outputs) > 0.5).float()
              total_val += labels.size(0)
              correct_val += (predicted == labels).sum().item()

      val_epoch_loss = val_running_loss / len(validation_dataset)
      val_epoch_acc = correct_val / total_val
      history['val_loss'].append(val_epoch_loss)
      history['val_acc'].append(val_epoch_acc)

      print(f"Epoch {epoch+1}/{num_epochs}, "
            f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
            f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")
  return history

In [64]:
hist = train_model()

Epoch 1/10, Loss: 0.6777, Acc: 0.4875, Val Loss: 0.6797, Val Acc: 0.4876
Epoch 2/10, Loss: 0.6480, Acc: 0.4875, Val Loss: 0.6608, Val Acc: 0.4876
Epoch 3/10, Loss: 0.6343, Acc: 0.4875, Val Loss: 0.6545, Val Acc: 0.4876
Epoch 4/10, Loss: 0.6330, Acc: 0.4875, Val Loss: 0.6531, Val Acc: 0.4876
Epoch 5/10, Loss: 0.6204, Acc: 0.4875, Val Loss: 0.6523, Val Acc: 0.4876
Epoch 6/10, Loss: 0.6127, Acc: 0.4875, Val Loss: 0.6506, Val Acc: 0.4876
Epoch 7/10, Loss: 0.6080, Acc: 0.4875, Val Loss: 0.6606, Val Acc: 0.4876
Epoch 8/10, Loss: 0.6059, Acc: 0.4875, Val Loss: 0.6535, Val Acc: 0.4876
Epoch 9/10, Loss: 0.5952, Acc: 0.4875, Val Loss: 0.6504, Val Acc: 0.4876
Epoch 10/10, Loss: 0.5937, Acc: 0.4875, Val Loss: 0.6991, Val Acc: 0.4876


In [65]:
np.median(hist["acc"])

np.float64(0.4875)

In [66]:
np.std(hist["loss"])

np.float64(0.024707762345359856)

In [67]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
  transforms.RandomRotation(50),
  transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
  transforms.RandomHorizontalFlip()
])

hist = train_model()

Epoch 1/10, Loss: 0.5954, Acc: 0.4875, Val Loss: 0.6528, Val Acc: 0.4876
Epoch 2/10, Loss: 0.5933, Acc: 0.4888, Val Loss: 0.6582, Val Acc: 0.4876
Epoch 3/10, Loss: 0.5816, Acc: 0.4875, Val Loss: 0.6611, Val Acc: 0.4876
Epoch 4/10, Loss: 0.5766, Acc: 0.4875, Val Loss: 0.6519, Val Acc: 0.4876
Epoch 5/10, Loss: 0.5748, Acc: 0.4888, Val Loss: 0.6727, Val Acc: 0.4876
Epoch 6/10, Loss: 0.5709, Acc: 0.4888, Val Loss: 0.6652, Val Acc: 0.4925
Epoch 7/10, Loss: 0.5673, Acc: 0.4913, Val Loss: 0.6496, Val Acc: 0.4975
Epoch 8/10, Loss: 0.5668, Acc: 0.4913, Val Loss: 0.6622, Val Acc: 0.4925
Epoch 9/10, Loss: 0.5651, Acc: 0.4938, Val Loss: 0.6487, Val Acc: 0.5025
Epoch 10/10, Loss: 0.5576, Acc: 0.4925, Val Loss: 0.6641, Val Acc: 0.4876


In [68]:
np.mean(hist["val_loss"])

np.float64(0.6586589408899421)

In [69]:
np.average(hist["val_acc"])

np.float64(0.491044776119403)